In [1]:
import sys
from pathlib import Path

In [2]:
import os
from pymongo import MongoClient

In [3]:
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

In [4]:
uri = os.environ['MONGO_URI']
mongo_client = MongoClient(uri)
mongo_client.server_info()

{'version': '4.0.5',
 'gitVersion': '3739429dd92b92d1b0ab120911a23d50bf03c412',
 'targetMinOS': 'Windows 7/Windows Server 2008 R2',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 0, 5, 0],
 'openssl': {'running': 'Windows SChannel'},
 'buildEnvironment': {'distmod': '2008plus-ssl',
  'distarch': 'x86_64',
  'cc': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'ccflags': '/nologo /EHsc /W3 /wd4355 /wd4800 /wd4267 /wd4244 /wd4290 /wd4068 /wd4351 /wd4373 /we4013 /we4099 /we4930 /WX /errorReport:none /MD /O2 /Oy- /bigobj /utf-8 /Zc:rvalueCast /Zc:strictStrings /volatile:iso /Gw /Gy /Zc:inline',
  'cxx': 'cl: Microsoft (R) C/C++ Optimizing Compiler Version 19.00.24223 for x64',
  'cxxflags': '/TP',
  'linkflags': '/nologo /DEBUG /INCREMENTAL:NO /LARGEADDRESSAWARE /OPT:REF',
  'target_arch': 'x86_64',
  'target_os': 'windows'},
 'bits': 64,
 'debug': False,
 'maxBsonObjectSize': 16777216,
 '

In [5]:
database = mongo_client['sms']

In [6]:
database.list_collection_names()

['import_entidades_pb',
 'usuarios_sei',
 'habilitacao',
 'fix_num_estacao_filtrado',
 'srd',
 'manifestacao',
 'cnpj_aid',
 'import_equipamentos',
 'licenciamento',
 'PH7apps',
 'habilitacaoTemp',
 'fix_data_validade',
 'emission',
 'municipio_ppdur',
 'entidade',
 'Licenciamento',
 'scr_licencas',
 'fix_num_estacao',
 'licenciamento_bloco',
 'import_documentos',
 'ato',
 'application',
 'fix_num_servico',
 'fix_datavalidade_175',
 'import_potencia_irradiada',
 'licenciamento_exclusao',
 'import_estacoes',
 'ato_velho',
 'fix_compara_emission',
 'municipio',
 'edital',
 'equipamento_tecnologia',
 'licenciamento_credito',
 'app',
 'fix_ato',
 'slp',
 'aero',
 'import_radcom_satelite',
 'log']

In [7]:
MONGO_SMP = {
    "$and": [
        {"DataExclusao": None},
        {"DataValidade": {"$nin": ["", None]}},
        {"Status.state": "LIC-LIC-01"},
        {"NumServico": "010"}, #{"$nin": ["010", "045", "171", "450", "750", "", None]}},
        {"FreqCentralMHz": {"$nin": [None, "", 0]}},
        {"CodMunicipio": {"$nin": [None, ""]}},
        {"FreqCentralMHz": {"$type": 1.0}},
        {"Latitude": {"$type": 1.0}},
        {"Longitude": {"$type": 1.0}},
    ]
}

COLS_SMP = {
    "NumAto": "Num_Ato",
    "NumFistel": "Fistel",
    "NumServico": "Num_Serviço",
    "NomeEntidade": "Entidade",
    "SiglaUf": "UF",
    "NumEstacao": "Número_Estação",
    "CodTipoClasseEstacao": "Classe",
    "NomeMunicipio": "Município",
    "CodMunicipio": "Código_Município",
    "DataValidade": "Validade_RF",
    "FreqCentralMHz": "Frequência_Central",
    "Latitude": "Latitude",
    "Longitude": "Longitude",
    "DesignacaoEmissao": "Designacao_Emissão",
    # "PotenciaTransmissorWatts" : "Potência(W)",
    # "CodTipoAntena": "Cod_TipoAntena",
    # "Polarizacao": "Polarização",
    # "RaioAntena": "Raio_Antena",
    # "GanhoAntena": "Ganho_Antena",
    # "FrenteCostaAntena": "FC_Antena",
    # "AnguloMeiaPotenciaAntena": "Ang_MP_Antena",
    # "AnguloElevacao": "Ângulo_Elevação",
    # "Azimute": "Azimute",
    # "AlturaAntena": "Altura_Antena",
    # "PerdasAcessorias": "Perdas_Acessorias",
}


In [8]:
collection = database["licenciamento"]

In [12]:
query_smp = collection.find(MONGO_SMP, limit=100000) #projection={k: 1.0 for k in COLS_SMP.keys()}

In [10]:
from tqdm.auto import tqdm

In [13]:
query_smp.next()

StopIteration: 

In [17]:
smp = list(tqdm(query_smp))

0it [00:00, ?it/s]

In [14]:
smp

NameError: name 'smp' is not defined

In [13]:
import gc
import pandas as pd
df = pd.DataFrame(smp, dtype='string')
del smp
gc.collect()
df.sample(5)

ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
df.to_parquet(Path.cwd().parent / 'dados' / 'smp.parquet.gzip', compression='gzip', index=False)

In [ ]:
df.groupby(['SiglaUf', 'NomeMunicipio', 'Latitude', 'Longitude', 'NumFistel',
            'FreqCentralMHz']).size().to_frame(name = 'Contagem_Fistel')

Contagem_Fistel
SiglaUf NomeMunicipio  Latitude Longitude NumFistel   FreqCentralMHz                 
AC      RIO BRANCO     -9.87014 -67.9121  50409889580 1822.5                        5
                                                      1875                          5
                                                      2130                          3
                                                      2640                          3
                                                      885                           3
...                                                                               ...
TO      PEDRO AFONSO   -8.97436 -48.17339 50409146366 1855                          3
                                                      874.5                         3
        PORTO NACIONAL -10.7101 -48.4089  50409889580 1822.5                        3
                                                      1875                          3
                                                      953.75                        2

[23457 rows x 1 columns]

In [ ]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(df, explorative=True)
# profile.to_notebook_iframe()